In [1]:
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.endpoints.trades import TradeList, TradeDetails
from oandapyV20.endpoints.transactions import TransactionList
from oandapyV20.endpoints.orders import OrderList
from oandapyV20.exceptions import V20Error

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

def fetch_orders(api, account_id):
    try:
        # Fetch open positions
        open_trades_request = TradeList(accountID=account_id)
        open_trades_response = api.request(open_trades_request)

        # Fetch orders
        orders_request = OrderList(accountID=account_id)
        orders_response = api.request(orders_request)

        return open_trades_response['trades'], orders_response['orders']
    except V20Error as e:
        logging.error(f"Error fetching orders and positions: {e}")
        return [], []

def format_orders_and_positions(open_trades, orders):
    data = []

    for trade in open_trades:
        trade_details_request = TradeDetails(accountID=account_id, tradeID=trade['id'])
        trade_details_response = api.request(trade_details_request)

        trade_data = trade_details_response['trade']
        order_data = {
            'Order Number': trade_data['id'],
            'Instrument': trade_data['instrument'],
            'Entry Price': trade_data['price'],
            'Current Price': trade_data['price'],
            'Stop Loss': trade_data['stopLossOrder']['price'] if 'stopLossOrder' in trade_data else None,
            'Take Profit': trade_data['takeProfitOrder']['price'] if 'takeProfitOrder' in trade_data else None,
            'Profit/Loss': trade_data['unrealizedPL'],
            'Open Time': trade_data['openTime'],
            'Close Time': None  # This will be None for open positions
        }
        data.append(order_data)

    for order in orders:
        order_data = {
            'Order Number': order['id'],
            'Instrument': order['instrument'],
            'Entry Price': order['price'],
            'Current Price': order['price'],
            'Stop Loss': order['stopLossOnFill']['price'] if 'stopLossOnFill' in order else None,
            'Take Profit': order['takeProfitOnFill']['price'] if 'takeProfitOnFill' in order else None,
            'Profit/Loss': 0,  # P/L will be calculated once the order is filled
            'Open Time': order['createTime'],
            'Close Time': None  # This will be updated when the order is filled/closed
        }
        data.append(order_data)

    df = pd.DataFrame(data)
    return df

def display_orders_table():
    open_trades, orders = fetch_orders(api, account_id)
    orders_df = format_orders_and_positions(open_trades, orders)
    print(orders_df)

# Call the function to display the orders table
display_orders_table()


ImportError: cannot import name 'TradeList' from 'oandapyV20.endpoints.trades' (d:\OneDrive\myproject\chatgpt-trader\.venv\Lib\site-packages\oandapyV20\endpoints\trades.py)